In [2]:
from utils import back_translate
from utils import compute_bleu_chrf
import json
file_path = "/home/mshahidul/project1/all_tran_data/Sampled_100_MedlinePlus_eng_spanish_pair.json"
with open(file_path, 'r', encoding='utf-8') as json_file:
    original_file = json.load(json_file)

In [ ]:

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
                    ### Instruction:
                    {}

                    ### Input:
                    {}

                    ### Response:
                    {}"""
max_seq_length=2048
model_name = "unsloth/DeepSeek-R1-Distill-Qwen-14B"
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = model_name, # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = None,
        load_in_4bit = False,
    )
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.4: Fast Qwen2 patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla V100-DGXS-32GB. Max memory: 31.727 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1. CUDA: 7.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [3]:
total_score=[]

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

def inference(ques):
    inputs = tokenizer(
    [
        alpaca_prompt.format(
            "Translate the input into English to Spanish:", # instruction
            ques, # input
            "", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
    ans=tokenizer.batch_decode(outputs)
    start_marker = '### Response:\n'
    end_marker = '<|end_of_text|>'

    # Find the start and end positions
    start_index = ans[0].find(start_marker) + len(start_marker)
    end_index = ans[0].find(end_marker)
    response = ans[0][start_index:end_index].strip()
    return response 


import tqdm
for line in tqdm.tqdm(original_file):
    hypothesis_text = inference(line['english'])
    reference_text = line['spanish']
    score=compute_bleu_chrf(reference_text, hypothesis_text)  
    total_score.append({
        "original_english": line['english'],
        "original_spanish": line['spanish'],
        "translated_spanish": hypothesis_text,
        "bleu_score": score
    })

In [ ]:
tt=model_name.split("/")[1]
avg_bleu_score = sum([x['bleu_score']['bleu_score'] for x in total_score]) / len(total_score)

print(f"{tt} without finetune --> Average BLEU Score: {avg_bleu_score:.4f}")

In [ ]:

with open(f"/home/mshahidul/project1/results/{tt}_without_finetune.json", 'w', encoding='utf-8') as json_file:
    json.dump(total_score, json_file, 9=False, indent=4)